In [1]:
import weka.core.jvm as jvm
from weka.core.converters import Loader, Saver
from weka.core.dataset import Instances
from weka.filters import Filter
jvm.start()

# Load the dataset
loader = Loader(classname="weka.core.converters.ArffLoader")
data = loader.load_file("z_listy_7.arff")
data.class_is_last()  # 

# C - attribute index
# L - value to delete ( 'first' or 'last' or specific value)
# V - reverse the selection
# S - split point
# R - attribute to remove
# first - remove instances with the first value
# last - remove instances with the last value
print("Number of instances before filtering(odmowa):", data.num_instances)

remove_filter = Filter(classname="weka.filters.unsupervised.instance.RemoveWithValues", options=[
    "-C", str(data.attribute_by_name("status pozyczki").index + 1), # index of the attribute
    "-L", "2"]) # value to remove
remove_filter.inputformat(data)
data_after_odmowa = Instances.copy_instances(remove_filter.filter(data))

print("Number of instances after filtering(odmowa):", data_after_odmowa.num_instances)

print("Number of instances before filtering(900):", data_after_odmowa.num_instances)
remove_filter = Filter(classname="weka.filters.unsupervised.instance.RemoveWithValues", options=[
    "-C", str(data_after_odmowa.attribute_by_name("kwota kredytu").index + 1), 
    "-S", "500", "-L", "last"]) 

remove_filter.inputformat(data_after_odmowa)
data_after_900 = Instances.copy_instances(remove_filter.filter(data_after_odmowa))
print("Number of instances after filtering(900):", data_after_900.num_instances) # powinno byc 65-9=56 wedlug csv

saver = Saver(classname="weka.core.converters.ArffSaver")
saver.save_file(data_after_900, "po_skrypcie_1.arff")

# remove_attribute = Filter(classname="weka.filters.unsupervised.attribute.Remove", options=["-R", str(data.attribute_by_name("status pozyczki").index + 1)])
# remove_attribute.inputformat(data)
# data = Instances.copy_instances(remove_attribute.filter(data))
# 
# saver = Saver(classname="weka.core.converters.ArffSaver")
# saver.save_file(data, "po_skrypcie_2.arff")
# 

jvm.stop()


DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\core.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\mtj.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


Number of instances before filtering(odmowa): 1324
Number of instances after filtering(odmowa): 1201
Number of instances before filtering(900): 1201
Number of instances after filtering(900): 337
